In [2]:
import argparse
import sys
import logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s %(message)s')
from gtfparse import read_gtf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from pyfaidx import Fasta

2019-05-02 22:26:53,352 font search path ['/opt/miniconda3/envs/py37/lib/python3.6/site-packages/matplotlib/mpl-data/fonts/ttf', '/opt/miniconda3/envs/py37/lib/python3.6/site-packages/matplotlib/mpl-data/fonts/afm', '/opt/miniconda3/envs/py37/lib/python3.6/site-packages/matplotlib/mpl-data/fonts/pdfcorefonts']
2019-05-02 22:27:00,467 generated new fontManager


In [3]:
def rank_transcripts(group, ranks):
    group['rank_score'] = group['tag'].str.split(",").apply(lambda x: sum(list(filter(None.__ne__, list(map(ranks.get, x))))))
    max_rank = group[group['rank_score'] == group['rank_score'].max()]
    return group.loc[(max_rank['end'] - max_rank['start']).idxmax()]

In [4]:
def extract_transcripts(df, biotype):
    """

    :param df:
    :param biotype:
    :return:
    """
    ranks = defaultdict(lambda: 0,
            {'MANE_Select': 10,
             'CCDS': 9,
             'appris_principal_1': 8,
             'appris_principal_2': 7,
             'appris_principal_3': 6,
             'appris_principal_4': 5,
             'appris_principal_5': 4,
             'basic': 3,
             'appris_alternative_1': 2,
             'appris_candidate': 1
             }
            )

    genes = df.loc[(df['gene_type']) == biotype]
    logging.info("#genes with {} biotype:\t{}".format(biotype,genes['gene_id'].nunique()))
    ranked_transcripts = genes.loc[(genes['transcript_type'] == biotype) & (genes['feature'] == "transcript")].\
        groupby('gene_id').apply(rank_transcripts, ranks)

    logging.info("#canonical transcripts retrieved (less than the #genes because all transcripts of some genes may lack"
                 " the biotype {}:\t{}".format(biotype, ranked_transcripts.shape[0]))
    return ranked_transcripts.set_index('transcript_id', append=True)

In [5]:
def retrieve_fasta_sequences(x, ref=None):

    if x['strand'] == "-":
        x['fasta'] = ref[x['seqname']][x['start']:x['end']].reverse.complement.seq
    else:
        x['fasta'] = ref[x['seqname']][x['start']:x['end']].seq
    return x

In [6]:
def mask_first_row(x):
    result = np.ones_like(x)
    result[0] = 0
    return result

In [7]:
def mask_last_row(x):
    result = np.ones_like(x)
    result[-1] = 0
    return result

In [10]:
def get_splice_sites(df, df_transcripts):
    """
    Produces a transformed dataframe containing all the splice sites of canonical transcripts extracted from
    exon coordinates, and generates bed files of those.

    Intercepts df of the whole GTF file with the df representative of canonical transcripts. This subset is used
    for the downstream operations, where first 2 intronic coordinates are retrieved and first/last exons are masked
    :param df:
    :param df_transcripts:
    :return: Dataframe of the exons of canonical transcripts to be used by other methods
    """

    logging.info("Extracting splice site coordinates from canonical transcripts..")
    ids = df_transcripts.index.get_level_values('transcript_id').tolist()
    exons = df.loc[(df['transcript_id'].isin(ids)) & (df['feature'] == "exon")]
    exons_to_bed = exons.sort_values(['seqname', 'start', 'end']).copy()

   # print(exons_to_bed.groupby("transcript_id").size())
   # print(exons_to_bed.loc[(exons['transcript_id'] == "ENST00000304952.10")][['transcript_id', 'start', 'end', 'gene_name','exon_number']])
    exons_to_bed.loc[:, 'before_start'], exons_to_bed.loc[:, 'after_end'], exons_to_bed.loc[:, 'start'] = \
        exons_to_bed.start - 3, exons_to_bed.end + 2, exons_to_bed.start - 1

    start = pd.melt(exons_to_bed, id_vars=["seqname", "before_start", "start", "gene_name", "exon_number", "strand"],
                    value_vars="transcript_id")

    mask_start = start.groupby(['value'])['value'].transform(mask_first_row).astype(bool)
    start = start.loc[mask_start]
    start.columns = ["chr", "start", "end", "gene", "exon_number", "strand", "group", "transcript_id"]

    end = pd.melt(exons_to_bed, id_vars=["seqname", "end", "after_end", "gene_name", "exon_number", "strand"],
                  value_vars="transcript_id")
    mask_end = end.groupby(['value'])['value'].transform(mask_last_row).astype(bool)
    end = end.loc[mask_end]
    end.columns = ["chr", "start", "end", "gene", "exon_number", "strand", "group", "transcript_id"]

    final_bed = pd.concat([start, end], axis=0)
    final_bed.sort_values(['chr', 'start', 'end'], inplace=True)
    del final_bed['group']
    final_bed.to_csv("canonical_splice_sites.bed.gz", compression="gzip", sep="\t", index=False, header=False)
    return final_bed

In [439]:
df = pd.read_csv('gtf_pandas.csv.gz')

In [440]:
canonical_transcripts = extract_transcripts(df, "protein_coding")

2019-05-04 22:42:34,323 #genes with protein_coding biotype:	20015
2019-05-04 22:43:37,213 #canonical transcripts retrieved (less than the #genes because all transcripts of some genes may lack the biotype protein_coding:	19747


In [375]:
def set_class(x):
    #arr = np.zeros(len(x['fasta']))
    #if positive strand, first splice site is an donor 
    if x['strand'] == "+":
        y = np.array([1, 2])
        l_y = np.tile(y, len(x['ss_idx']))
    
    #if negative strand, first splice site is an acceptor, last intron (coordinates based)
    elif x['strand'] == "-":
        y = np.array([2, 1])
        l_y = np.tile(y, len(x['ss_idx']))
    
    np.put(x['class'], x['ss_idx'], l_y)
    np.put(x['class'], x['ss_idx'] + 1, l_y)

In [423]:
def get_training_data(transcripts, ss, fastafile):
    logging.info("Retrieving transcripts sequences")
    ref_genome = Fasta(fastafile)
    
    #add fasta column to each row of transcripts df
    fasta = transcripts.apply(retrieve_fasta_sequences, axis=1, ref=ref_genome)
    
    #transcripts start coord
    transcripts_coord = transcripts['start'].reset_index(level=0, drop=True).to_dict()
    
    #get exons coord from the beginning of the transcript
    ss['idx_start'] = ss['start'] - list(map(transcripts_coord.get, ss['transcript_id']))
    
    #get idx where ss are within each transcript
    per_transcript_start_idx = ss.groupby('transcript_id')['idx_start'].apply(np.hstack)
    
    #merge indexes to fasta df
    fasta=fasta.merge(per_transcript_start_idx.rename('ss_idx').to_frame(), left_on="transcript_id", right_index=True)
    
    #create class array filled with zeros
    fasta['class'] = fasta['fasta'].apply(lambda x: np.zeros(len(x), dtype=int))
    
    #set class splice donors (1) and splice acceptor (2)
    fasta.apply(set_class, axis =1)
    fasta['Y'] = fasta['class'].apply(lambda x: ''.join(map(str,x)))

    array = fasta[['fasta','Y']].values.reshape(-1,2)
   #print(array[:,0:1])
    

In [441]:
%%time
canonical_exons = get_splice_sites(df, canonical_transcripts) 

2019-05-04 22:43:37,468 Extracting splice site coordinates from canonical transcripts..
CPU times: user 17.2 s, sys: 0 ns, total: 17.2 s
Wall time: 14.8 s


In [442]:
%%time
get_training_data(canonical_transcripts, canonical_exons, "/home/pbarbosa/Desktop/GRCh38.primary.genome.fa")

2019-05-04 22:43:52,284 Retrieving transcripts sequences
[['...']]
CPU times: user 13min 47s, sys: 2min 18s, total: 16min 6s
Wall time: 20min 13s


In [438]:
x = np.arange(12)
a = np.array("ATGTGACGTCAGTCGACTAGCATGCATGCTA0000000000001100000000000000220")
b = np.array("0000000000001100000000000000220")
#c = np.add(a,b)
c = a.reshape(-1,2)
print(type(c))
print(x[1])
x.shape = (3,4) 
print(x)
print(x[:,100:105])
print(c)

ValueError: cannot reshape array of size 1 into shape (2)